In [1]:
from core.vsm_visualizer import VectorSpaceModel
from core.query import QueryHandler
import ir_datasets
import ir_measures
from ir_measures import nDCG, P, Judged, R, AP, RR,AP_IA
import pandas as pd
import json

## Initializing Datasets and Variables

In [2]:
dataset = ir_datasets.load("wikir/en1k/test")

vsm = VectorSpaceModel.load("wiki_state.pkl")
query_handler = QueryHandler(vsm)

### Initializing Queries and storing the results

In [3]:
# Get queries
queries = dataset.queries_iter()

query_rows = []
for index, (query_id, query_text) in enumerate(queries):
    # search for the query
    search_results = query_handler.search(query_text,similarity_threshold= 0.001)
    for (document, similarity) in search_results:
        new_row = {'query_id': query_id,
                   'doc_id': document[0], 
                   'score': similarity}
        query_rows.append(new_row)


query_table_results = pd.DataFrame(
    query_rows, columns=['query_id', 'doc_id', 'score'])

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'colombia', 'cri', 'describ', 'egypt', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'germani', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'ireland', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'peru', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'state', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'unit', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  warnings.warn(


### Add the qrels into dataframe

In [4]:
# get the qrels into dataframe
qrels_rows = []

for qrel in dataset.qrels_iter():
    new_row = {'query_id': qrel.query_id,
                   'doc_id': qrel.doc_id, 'relevance': qrel.relevance}
    qrels_rows.append(new_row)

qrels_table_results = pd.DataFrame(
    qrels_rows, columns=['query_id', 'doc_id', 'relevance'])

### Claculate The Evaluation Results

In [5]:
ir_measures.calc_aggregate(
    [
    AP(rel=1),
    AP(rel=2),
    P(rel=1)@10,
    R(rel=1)@10,
    RR(rel=1)], 
    qrels_table_results,
    query_table_results
    )

{AP: 0.1592827558164802,
 R@10: 0.18194934268357665,
 AP(rel=2): 0.5356769255138595,
 RR: 0.6231084940743323,
 P@10: 0.19200000000000003}

{AP: 0.1592827558164802,
 R@10: 0.18194934268357665,
 AP(rel=2): 0.5356769255138595,
 RR: 0.6231084940743323,
 P@10: 0.19200000000000003}